In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(r"db\market_place.db")

cursor = conn.cursor()

In [2]:
aisles = pd.read_csv(r"data\aisles.csv", sep=",")
departments = pd.read_csv(r"data\departments.csv", sep=",")
products = pd.read_csv(r"data\products.csv", sep=",")
orders = pd.read_csv(r"data\orders_with_nulls.csv", sep=",")

print("NULOS POR TABLA")
print(aisles.isna().sum(), "\n")
print(departments.isna().sum(), "\n")
print(products.isna().sum(), "\n")
print(orders.isna().sum(), "\n")

print("VALORES UNICOS EN ID")
print(aisles['aisle_id'].value_counts()[aisles['aisle_id'].value_counts() > 1])
print(departments['department_id'].value_counts()[departments['department_id'].value_counts() > 1])
print(products['product_id'].value_counts()[products['product_id'].value_counts() > 1])

NULOS POR TABLA
aisle_id      0
aisle_name    0
dtype: int64 

department_id      0
department_name    0
dtype: int64 

product_id       0
product_name     0
aisle_id         0
department_id    0
dtype: int64 

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64 

VALORES UNICOS EN ID
Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)


In [3]:
# Calcular la media de 'days_since_prior_order' por usuario
mean_days_per_user = orders.groupby('user_id')['days_since_prior_order'].transform('mean')

# Reemplazar los valores NaN con la media de cada usuario
orders['days_since_prior_order'] = orders['days_since_prior_order'].fillna(mean_days_per_user).astype(int)

orders.to_csv("data\orders_not_nulls.csv", index=False)

CREAR UN UNIC ORDERS AMB PRIOR I TRAIN SENSE NULS

In [ ]:
orders_prior = pd.read_csv(r"data\order_products__prior.csv", sep=",")
orders_train = pd.read_csv(r"data\order_products__train.csv", sep=",")

order_products = pd.concat([orders_prior, orders_train], ignore_index=True)

# Hacer merge con la tabla de órdenes
merged = pd.merge(order_products, orders, on='order_id', how='left')

merged.to_csv('data/final_orders.csv', index=False)

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS aisles (
        aisle_id INT PRIMARY KEY,
        aisle_name TEXT NOT NULL
    )
    """
)

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS departments (
        department_id INT PRIMARY KEY,
        department_name TEXT NOT NULL
    )
    """
)

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS products (
        product_id INT PRIMARY KEY,
        product_name TEXT NOT NULL,
        aisle_id INT NOT NULL,
        department_id INT NOT NULL,
        FOREIGN KEY (aisle_id) REFERENCES aisles(aisle_id),
        FOREIGN KEY (department_id) REFERENCES departments(department_id)
    )
    """
)

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS users (
        user_id INT PRIMARY KEY,
        n_compras INT NOT NULL,
        last_order_date INT NOT NULL,
        mean_time_bt_orders INT NOT NULL,
        segment TEXT NOT NULL
    )
    """
)

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS orders (
        order_id INT NOT NULL,
        user_id INT NOT NULL,
        product_id INT NOT NULL,
        add_to_cart_order INT NOT NULL,
        reordered INT NOT NULL,
        order_number INT NOT NULL,
        order_dow INT NOT NULL,
        order_hour_of_day INT NOT NULL,
        days_since_prio_order INT NOT NULL,
        PRIMARY KEY (order_id,user_id,product_id),
        FOREIGN KEY (product_id) REFERENCES products(product_id)
    )
    """
)

CREATE TABLE IF NOT EXISTS svd_predictions (
    user_id INTEGER,
    product_id INTEGER,
    score REAL,    
    PRIMARY KEY (user_id, product_id)
)

conn.commit()

In [ ]:
# Inserta los datos en la base de datos
users = pd.read_csv(r"data\segmented_users.csv", sep=",")
#merged = pd.read_csv(r"data\final_orders.csv", sep=",")

# PASAR A MINUSCULAS TODOS LOS NOMBRES DE PRODUCTOS
products['product_name'] = products['product_name'].str.lower()
products['product_name'] = products['product_name'].str.replace('"', '', regex=False)
products['product_name'] = products['product_name'].str.replace(r"[^a-z0-9' ]", "", regex=True)
products = products.drop_duplicates(subset=['product_name', 'aisle_id', 'department_id'], keep='first')


aisles.to_sql("aisles", conn, if_exists="append", index=False)
departments.to_sql("departments", conn, if_exists="append", index=False)
products.to_sql("products", conn, if_exists="append", index=False)
users.to_sql("users", conn, if_exists="append", index=False)

chunk_size = 10000
total = len(merged)
for i in range(0, total, chunk_size):
    chunk = merged.iloc[i:i+chunk_size]
    chunk.to_sql("orders", conn, if_exists="append", index=False)
    print(f"Inserted rows {i} to {i + len(chunk) - 1}")




Inserted rows 0 to 9999
Inserted rows 10000 to 19999
Inserted rows 20000 to 29999
Inserted rows 30000 to 39999
Inserted rows 40000 to 49999
Inserted rows 50000 to 59999
Inserted rows 60000 to 69999
Inserted rows 70000 to 79999
Inserted rows 80000 to 89999
Inserted rows 90000 to 99999
Inserted rows 100000 to 109999
Inserted rows 110000 to 119999
Inserted rows 120000 to 129999
Inserted rows 130000 to 139999
Inserted rows 140000 to 149999
Inserted rows 150000 to 159999
Inserted rows 160000 to 169999
Inserted rows 170000 to 179999
Inserted rows 180000 to 189999
Inserted rows 190000 to 199999
Inserted rows 200000 to 209999
Inserted rows 210000 to 219999
Inserted rows 220000 to 229999
Inserted rows 230000 to 239999
Inserted rows 240000 to 249999
Inserted rows 250000 to 259999
Inserted rows 260000 to 269999
Inserted rows 270000 to 279999
Inserted rows 280000 to 289999
Inserted rows 290000 to 299999
Inserted rows 300000 to 309999
Inserted rows 310000 to 319999
Inserted rows 320000 to 329999
In

In [3]:
import joblib

usuarios = pd.read_sql_query("SELECT user_id, segmento FROM users", conn)
productos_top = pd.read_sql_query(
    "SELECT product_id FROM orders GROUP BY product_id ORDER BY COUNT(*) DESC LIMIT 500", conn
)['product_id'].tolist()

# Crear tabla si no existe
conn.execute("""
CREATE TABLE IF NOT EXISTS svd_predictions (
    user_id INTEGER,
    product_id INTEGER,
    score REAL,
    PRIMARY KEY (user_id, product_id)
)
""")

# Predecir y guardar
for segmento in usuarios['segmento'].unique():
    model = joblib.load(f"models/SVD/SVD_{segmento}.joblib")
    user_ids = usuarios[usuarios['segmento'] == segmento]['user_id']
    for user_id in user_ids:
        for pid in productos_top:
            score = model.predict(user_id, pid).est
            conn.execute(
                "INSERT OR REPLACE INTO svd_predictions (user_id, product_id, score) VALUES (?, ?, ?)",
                (user_id, pid, score)
            )
conn.commit()
conn.close()